#** Machine learning IB-2024** #** Home task 1.** #Regression, KNN, LinearRegression.

In this homework we will build models to predict the price of an apartment in Russia. below is a description of some columns of the data set. date - date of publication price - price in rubles level- floor, on which is the apartment levels - number of floors in the apartment rooms - number of rooms in the apartment. If value -1, then the apartment is considered apartments. area - area of the apartment. kitchen_area - area of the kitchen. geo_lat - Latitude geo_lon - Longitude building_type - building material. 0 - Don't know. 1 - Other. 2 - Panel. 3 - Monolithic. 4 - Brick. 5 - Blocky. 6 - Wooden

Part 0 – Start of work

To start working with the data, we import the libraries that will be needed in this task.

In [ ]:
import math
import pandas as pd
import numpy as np
import matplotlib as plt
import sklearn
import seaborn as sns

Download the folium library to display the data on the map by coordinates.

In [ ]:
!pip install folium

We delete our data from the archive.

In [ ]:
!unzip ...

Download the data from the CSV file into the data frame.

In [ ]:
df = pd.read_csv("input_data.csv", sep=";")

In [ ]:
df

,date,price,level,levels,rooms,area,kitchen_area,geo_lat,geo_lon,building_type,object_type,postal_code,street_id,id_region,house_id
0,2021-01-01,2451300,15,31,1,30.3,0.0,56.780112,60.699355,0,2,620000.0,NaN,66,1632918.0
1,2021-01-01,1450000,5,5,1,33.0,6.0,44.608154,40.138381,0,0,385000.0,NaN,1,NaN
2,2021-01-01,10700000,4,13,3,85.0,12.0,55.540060,37.725112,3,0,142701.0,242543.0,50,681306.0
3,2021-01-01,3100000,3,5,3,82.0,9.0,44.608154,40.138381,0,0,385000.0,NaN,1,NaN
4,2021-01-01,2500000,2,3,1,30.0,9.0,44.738685,37.713668,3,2,353960.0,439378.0,23,1730985.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119299,2021-01-11,1850000,3,10,1,37.2,8.5,53.018837,36.131930,0,0,302038.0,NaN,57,NaN
119300,2021-01-11,3990000,1,9,3,57.1,10.0,47.229338,39.730318,2,0,344022.0,328588.0,61,2007296.0
119301,2021-01-11,3500000,9,9,3,57.0,-100.0,47.229338,39.730318,2,0,344022.0,328588.0,61,2007296.0
119302,2021-01-11,3460000,7,9,3,63.0,6.6,64.533132,40.611343,2,0,163065.0,NaN,29,NaN


Show the coordinates of our buildings on the map.

In [ ]:
import folium
from IPython.display import display

map_df = df.loc[:1000]

m = folium.Map(location=[55.751244, 37.618423], zoom_start=10)

# List of points with width and length
lats = map_df['geo_lat'].loc[:1000]
longs = map_df['geo_lon'].loc[:1000]
# Adding points to the map
for point in zip(lats, longs):
    folium.Marker(
        location=[point[0], point[1]]
    ).add_to(m)

display(m)

Part 1: Preparing data for processing by machine learning models.

**0.5 Ball**. The geography of our observations in the data set is extremely large. However, we know that the cost of apartments in Moscow and St. Petersburg is much higher than the average in Russia. Let's make signs that show whether the apartment is 20 kilometers from the centre of Moscow or whether the apartment is 20 kilometers from the centre of St. Petersburg. Create two signs is_Moscow and is_Saint_Peterburg. To find the distance by the coordinates use the function haversine_distance.

In [ ]:
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in km
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)

    a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    return R * c

# Centre of Moscow and St. Petersburg
moscow_lat, moscow_lon = 55.751244, 37.618423
spb_lat, spb_lon = 59.93428, 30.335099

# Signs is_Moscow and is_Saint_Peterburg
df['is_Moscow'] = df.apply(lambda row: 1 if haversine_distance(row['geo_lat'], row['geo_lon'], moscow_lat, moscow_lon) < 20 else 0, axis=1)
df['is_Saint_Peterburg'] = df.apply(lambda row: 1 if haversine_distance(row['geo_lat'], row['geo_lon'], spb_lat, spb_lon) < 20 else 0, axis=1)


**0.5 Ball**. In our data set there are signs that we can theoretically use, such as postal_code, but we will do this within the framework of homework very-very long. Therefore it is proposed to remove unnecessary signs from the data frame. We will remove geo_lat, geo_lon, object_type, postal_code, street_id, id_region, house_id.

In [ ]:
df.drop(columns=['geo_lat', 'geo_lon', 'object_type', 'postal_code', 'street_id', 'id_region', 'house_id'], inplace=True)


What columns are categorical? What numbers are? Categorical: (Your answer) Numbers: (Your answer) Let us encode categorical characters using OneHot-Encoding. https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(drop='first', sparse_output=False)
encoded_building_type = encoder.fit_transform(df[['building_type']])

# Adding new columns to DataFrame
encoded_df = pd.DataFrame(encoded_building_type, columns=encoder.get_feature_names_out())
df = pd.concat([df, encoded_df], axis=1)
df.drop(columns=['building_type'], inplace=True)

**0.5 Ball**. We work with numerical characters: 1. Add two characters to your dataset: the number of days from the day of the first observation (the difference between the dates of the ads). Maybe for the price forecast the floor is not as important as the ratio of the floor to the number of floors in the house, add this character. After adding a new character the date column can be removed. 2. The numerical characters may have different orders. Let us compare the numerical characters using StandartScaller https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html.

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df['days_since_first'] = (df['date'] - df['date'].min()).dt.days
df['floor_ratio'] = df['level'] / df['levels']
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
numerical_columns = ['price', 'level', 'levels', 'rooms', 'area', 'kitchen_area', 'days_since_first', 'floor_ratio']

df[numerical_columns] = scaler.fit_transform(df[numerical_columns])


**2 Ball**. Realize the KNNRegressor class, which should make regression by the method to the nearest neighbors.

In [ ]:
class KNNRegressor:
    def __init__(self, n_neighbors=5, metric='euclidean'):
        self.n_neighbors = n_neighbors
        self.metric = metric

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        from scipy.spatial.distance import cdist
        distances = cdist(X, self.X_train, metric=self.metric)
        nearest_indices = np.argsort(distances, axis=1)[:, :self.n_neighbors]
        y_pred = np.mean(self.y_train[nearest_indices], axis=1)
        return y_pred


**3 Balls**. Realize the LinearRegression class that supports gradient leaks SGD, Momentum, AdaGrad. Use gradient to optimize the MSE loss function.

In [ ]:
class LinearRegression:
    def __init__(self, learning_rate=0.01, optimization='SGD', epsilon=1e-8, decay_rate=0.9, max_iter=1000):
        self.learning_rate = learning_rate
        self.optimization = optimization
        self.epsilon = epsilon
        self.decay_rate = decay_rate
        self.max_iter = max_iter
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        # Add a column of single values for shift
        X = np.c_[np.ones(X.shape[0]), X]
        m = X.shape[0]
        n = X.shape[1]

        self.weights = np.zeros(n)
        self.bias = 0

        for i in range(self.max_iter):
            # Calculation of Predictions
            y_pred = X.dot(self.weights) + self.bias
            error = y_pred - y

            # Gradients
            d_weights = (2/m) * X.T.dot(error)
            d_bias = (2/m) * np.sum(error)

            # Updated Weights
            self.weights -= self.learning_rate * d_weights
            self.bias -= self.learning_rate * d_bias

    def predict(self, X):
        X = np.c_[np.ones(X.shape[0]), X]  # Add a column of single values for shift
        return X.dot(self.weights) + self.bias


Part 2: Experiments with machine learning models.

**3 Balls**. Do experiments with the machine learning methods you have written. Separate the training and test samples at 0.8 and 0.2 respectively. Measure the MSE, MAE, RMSE error. Use the KNNRegressor and LinearRegression methods from the sklearn library, compare the quality of your solutions and library.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import mean_squared_error

X = df.drop(columns=['price', 'date'])
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# by KNNRegressor
knn = KNNRegressor(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)
mse_knn = mean_squared_error(y_test, y_pred_knn)
mae_knn = mean_absolute_error(y_test, y_pred_knn)
rmse_knn = np.sqrt(mse_knn)

# Linear Regression (sklearn)
from sklearn.linear_model import LinearRegression as SklearnLR
sklearn_lr = SklearnLR()
sklearn_lr.fit(X_train, y_train)
y_pred_lr = sklearn_lr.predict(X_test)
mse_lr = mean_squared_error(y_test, y_pred_lr)
mae_lr = mean_absolute_error(y_test, y_pred_lr)
rmse_lr = np.sqrt(mse_lr)

